In [1]:
from IPython.display import display, HTML

display(
    HTML("""
<style>
div.container{width:80% !important;}
div.prompt {min-width:70px;}
div#toc-header{margin-top:150px;}
span.toc-item-num{display:none;}
div.CodeMirror {font-family:Consolas}
div.input {font-family:Consolas}
</style>
"""))

In [2]:
# 1. 데이터 확보
import zipfile
filepath = 'data/glaucoma.zip'
with zipfile.ZipFile(filepath, 'r') as f:
    f.extractall('data/')

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(
    rotation_range=20,       # 최대 회전 각도
    width_shift_range=0.2,   # 가로 방향 ±20% 이동 정도
    height_shift_range=0.2,  # 상하 방향 ±20% 이동 정도
    horizontal_flip=True,    # 수평 뒤집기
    vertical_flip=True)      # 수직 뒤집기
train_data = train_generator.flow_from_directory(
    'data/glaucoma/train/',
    target_size=(224, 224),
    batch_size=4,
    class_mode='sparse')  # 종속변수에 원핫인코딩 하지 않음
test_generator = ImageDataGenerator()
test_data = test_generator.flow_from_directory(
    'data/glaucoma/test/',
    target_size=(224, 224),
    batch_size=4,
    class_mode='sparse')

Found 828 images belonging to 3 classes.
Found 320 images belonging to 3 classes.


In [5]:
# 학습데이터와 테스트 데이터가 될 이미지의 채널 확인
img = image.load_img('data/glaucoma/test/normal_control/1.png')
img_data = image.img_to_array(img)
img_data.shape

(240, 240, 3)

In [4]:
# 2. 모델 구성
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import decode_predictions
import numpy as np
from tensorflow.keras.models import Sequential    # ResNet50의 Dense 층을 사용자 추가
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing import image  # 이미지 읽어오기

resnet_model = ResNet50(
    input_shape=(224, 224, 3),
    include_top=False)  # 컨볼루션과 폴링 레이어만 구조와 가중치를 가져옴(최상위 Dense층 제외)
resnet_model.trainable = True  # ResNet의 가중치도 업데이트 가능
# model = Sequential()
# model.add(resnet_model)
# model.add(Flatten())
# model.add(Dense(units=1024, activation='relu'))
# model.add(Dense(units=3,    activation='softmax'))
model = Sequential([
    resnet_model,
    Flatten(),
    Dense(units=1024, activation='relu'),
    Dense(units=3,    activation='softmax')
])
model.summary()
# 학습 시 필요한 용량
#  1. 파라미터 사용 메모리 사용량 : 파라미터수(126,352,259)*4byte / 1024의 제곱 = 482MB
#  2. 배치사이즈=3일 경우 3 * (224*224*3) * 4byte / 1024의 제곱 = 19MB
# 각 layer층 마다 활성화 값

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1024)              102761472 
                                                                 
 dense_1 (Dense)             (None, 3)                 3075      
                                                                 
Total params: 126,352,259
Trainable params: 126,299,139
Non-trainable params: 53,120
_________________________________________________________________


In [6]:
# 3. 모델 학습설정
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 학습시키기
model.fit(train_data, validation_data=test_data, epochs=20, batch_size=1)

Epoch 1/20
207/207 [==============================] - 637s 3s/step - loss: 6.6807 - accuracy: 0.5785 - val_loss: 3.6147 - val_accuracy: 0.6844
Epoch 2/20
131/207 [=================>............] - ETA: 3:44 - loss: 0.9630 - accuracy: 0.5954

KeyboardInterrupt: 